In [ ]:
# package for managing experiments
from sad2_final_project.analysis import BooleanNetworkExperiment
from sad2_final_project.boolean_bn.bn import BN
from sad2_final_project.boolean_bn.bn_sampling import simulate_trajectories_to_csv
from sad2_final_project.bnfinder import manager_bnfinder
from sad2_final_project.boolean_bn import load_bnet_to_BN

# system packages
import os
from pathlib import Path
import numpy as np

cwd=Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent) 
print(os.getcwd())

## create paths 
DATA_PATH = Path('data')
MODELS_PATH = Path("download_models")

In [ ]:
def run_single_condition(bn: BN, score_functions, data_path, experiment_name) -> bool:
    """
    Executes ONE experimental condition.
    This function is process-safe.
    """


    # sampling frequency
    if bn.mode == 'synchronous':
        sampling_frequency = 1
    elif bn.mode == 'asynchronous' and score_functions == ['MDL']:
        sampling_frequency = 2 
    else:
        sampling_frequency = 3
    
    #  k value
    if bn.mode == 'asynchronous':
        k =100
    elif bn.mode == "synchronous" and bn.num_nodes <= 5:
        k=20
    else:
        k=40

    # trajectory_length
    trajectory_length = int(round(bn.num_nodes * 0.8))
    n_trajectories = int(round((bn.num_nodes * k)/(trajectory_length)))

    # ---------- paths ----------
    gt_path = data_path / experiment_name / "bn_ground_truth" / f"{experiment_name}.csv"
    dataset_path = data_path / experiment_name / "datasets" / f"{experiment_name}.csv"
    bnf_dataset_path = data_path / experiment_name / "datasets_bnfinder" / f"{experiment_name}.txt"
    metrics_path = data_path / experiment_name / "results" / f"{experiment_name}_metrics.csv"

    # ---------- 2. save ground truth ----------
    bn.save_ground_truth(gt_path)

    # ---------- 3. generate data ----------
    # oznaczamy czy wygenerowany dataset powiódł sie sukcesem (potrzebne do zliczania sukcesów i jeżeli brak sukcesu nie próbujemy uruchamiać bnfindera bo brak pliku)
    success, ratio = simulate_trajectories_to_csv(
        bn_instance=bn,
        num_trajectories=n_trajectories,
        output_file=dataset_path,
        sampling_frequency=sampling_frequency,
        trajectory_length=trajectory_length,
        target_attractor_ratio = 0.5,
        tolerance = 0.5
    )

    # if not success:
    #     print(f"[INFO] Dataset {dataset_path} NOT created (requirements not met). Skipping inference.")
    #     return False

    # ---------- 4. inference ----------
    manager_bnfinder(
        ## file paths
        dataset_path=dataset_path,
        ground_truth_path=gt_path,
        trained_model_name= data_path / experiment_name / "results" / experiment_name,
        bnf_file_path=bnf_dataset_path,
        metrics_file=metrics_path,
        ## model parameters
        score_functions=score_functions,
        ## analysis settings
        #analysis_metrics=self.analysis_metrics,
        #analysis_score_functions=self.analysis_score_functions,
        dataset_succeeded=success,
        attractor_ratio=ratio,
    )

    return True

In [ ]:
MODEL_NAME = "model_id_37"

# loading model
bn = load_bnet_to_BN(MODELS_PATH / MODEL_NAME / "model.bnet")
# train model with respect to score function
for score_function in ['MDL', 'BDE']:
    ## create directories 
    !mkdir {DATA_PATH}
    !mkdir {DATA_PATH / (MODEL_NAME + '_' + score_function)}
    !mkdir {DATA_PATH / (MODEL_NAME + '_' +  score_function) / "results"}
    !mkdir {DATA_PATH / (MODEL_NAME + '_' +  score_function) / "datasets"}
    !mkdir {DATA_PATH / (MODEL_NAME + '_' +  score_function) / "bn_ground_truth"}
    !mkdir {DATA_PATH / (MODEL_NAME + '_' +  score_function) / "datasets_bnfinder"}
    if run_single_condition(bn, [score_function], DATA_PATH, MODEL_NAME + '_' + score_function):
        print(f"Experiment {MODEL_NAME} completed successfully.")
    else:
        print(f"Experiment {MODEL_NAME} failed.")